In [2]:
!pip install pixellib

     |████████████████████████████████| 430 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 985 kB 122.0 MB/s eta 0:00:01
     |████████████████████████████████| 13.3 MB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 18.2 MB/s eta 0:00:01
     |████████████████████████████████| 948 kB 19.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.0 MB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 24.0 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 15.7 MB/s eta 0:00:01
     |████████████████████████████████| 46.3 MB 6.9 MB/s eta 0:00:011
     |████████████████████████████████| 50 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 17.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 111.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 11.6 MB/s eta 0:00:01
     |█████████████████████████████

  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=87ac90a2edfa73dad907b6f0fdaa68dfdb9b6d6d272e4244204712cdf5ae6006
  Stored in directory: /Users/reza/Library/Caches/pip/wheels/d9/1b/13/b526842a9074220158b60dad35d854a13107c8a10e26eb1e50
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=350b283f11cc891a563bc804e00856d19ce1a568abae7031f6feb2ce5d9c7a13
  Stored in directory: /Users/reza/Library/Caches/pip/wheels/b6/0d/90/0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=8d3b5ab37c9debea922520798f0bd8516a6db0e3c4dc7c6c11ca0531f0cfa3f6
  Stored in directory: /Users/reza/Library/Caches/pip/wheels/23/cf/80/f3efa822e6ab23277902ee9165fe772eeb1dfb8014f359020a
  Created wheel for imantics: filename=imantics-0.1.12-py3-none-any.whl size=16033 sha256=998e39ee440da68c46c5b52f1524ae8d275303dfe353e9

In [4]:
import pixellib
from pixellib.semantic import semantic_segmentation
import os

folderPath = "/Users/reza/EVERYTHING/Work/DM\ Book/Codes/Ch12"
modelname = "deeplabv3_xception_tf_dim_ordering_tf_kernels.h5"

modelPath= os.path.join(folderPath, modelname)
print(modelPath)
segment_image = semantic_segmentation()
segment_image.load_pascalvoc_model("./deeplabv3_xception_tf_dim_ordering_tf_kernels.h5") 

segment_image.segmentAsPascalvoc("./rabbits.jpeg", output_image_name = "./rabbitsSEG.jpeg",overlay = True)

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
import pixellib
from pixellib.instance import instance_segmentation

segment_image = instance_segmentation()
segment_image.load_model("./mask_rcnn_coco.h5") 
segment_image.segmentImage("./Nassir.jpg", output_image_name = "./NassirSEG.jpg")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True)
model.eval()

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms

filename = "/Users/rawassizadeh/EVERYTHING/Work/DM Book/Codes/Ch12/rabbits.jpeg"

input_image = Image.open(filename)
input_image = input_image.convert("RGB")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

In [ ]:
# create a color pallette, selecting a color for each class
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# plot the semantic segmentation predictions of 21 classes in each color
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(input_image.size)
r.putpalette(colors)

import matplotlib.pyplot as plt
plt.imshow(r)
# plt.show()